In [2]:
from langchain.schema import Document

In [3]:
from langchain_chroma import Chroma

In [4]:
from langchain_ollama.embeddings import OllamaEmbeddings

In [5]:
from langchain_ollama import ChatOllama

In [6]:
from langchain.chains import RetrievalQA

In [8]:
from langchain.prompts import PromptTemplate

In [9]:
doc1 = Document(page_content="Virat Kohli (Batsman, Former Captain): Scored 8 centuries in 2024, ranking among the top Test and ODI batsmen globally.",
                metadata={"role":"Batsman, Former Captain"})

In [10]:
doc2 = Document(page_content="Rohit Sharma (Batsman, ODI/Test Captain): Led India to the 2024 T20 World Cup victory, ranked third in ICC ODI batting rankings",
                metadata={"role":"Batsman, ODI/Test Captain"})

In [11]:
doc3 = Document(page_content="Jasprit Bumrah (Bowler): Top-ranked Test bowler, leading India’s pace attack with 6/70 in the 2025 England Test series.",
                metadata={"role":"(Bowler)"})

In [12]:
docs=[doc1,doc2,doc3]

In [13]:
vector_store =Chroma(
   embedding_function=OllamaEmbeddings(model="granite-embedding:latest"),
   persist_directory="my_chroma_db",
   collection_name="sample"
)

In [14]:
vector_store.add_documents(docs)

['bda01f45-9ed6-449c-95db-00dafced05f5',
 '65848990-f59d-4cec-9965-9dfc9c264f40',
 'd0689b05-fc31-4f94-b038-4321d78ad30a']

In [ ]:
vector_store.get(include=["embeddings","documents","metadatas"])

In [15]:
llm = ChatOllama(model="llama3.2:latest")

In [16]:
prompt_template = """

You are a cricket expert. Answer the question based on the provided context.
Context: {context}

Question: {question}

Answer:

"""

In [17]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [24]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [25]:
query = "Who among are the top bowlers?"

In [26]:
result = qa_chain.invoke({"query": query})

In [ ]:
print("Answer:", result['result'])
print(" \nDocument used :", result['source_documents'][0].page_content)

Answer: Based on the provided context, Jasprit Bumrah is currently considered the top-ranked Test bowler.
 
 Document used : Jasprit Bumrah (Bowler): Top-ranked Test bowler, leading India’s pace attack with 6/70 in the 2025 England Test series.
